In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from fake_useragent import UserAgent
from selenium.webdriver.support import expected_conditions as EC


op = webdriver.ChromeOptions()
op.add_argument(f"user-agent={UserAgent.random}")
op.add_argument("user-data-dir=./")
op.add_experimental_option("detach", True)
op.add_experimental_option("excludeSwitches", ["enable-logging"])

driver = uc.Chrome(chrome_options=op)
driver.get('https://chat.openai.com/auth/login')

sleep(3)

inputElements = driver.find_elements(By.TAG_NAME, "button")
inputElements[0].click()

sleep(3)



In [6]:
import os

In [19]:
file_paths = [
"<PDF 경로들"

]

In [20]:
print(len(file_paths))

16


In [21]:
    # 프롬프트 입력
prompt_text = prompt_gem = """<문제 생성 예시>
\n
Question : 
\n
Option:
\n
A.
\n
B.
\n
C.
\n
D.
\n
Answer:
\n
Type(문제유형) : 객관식 문제
\n
Page(문제가 만들어진 페이지) :
\n
================================================================================
\n
Question :
\n
Option:
\n
A.
\n
B.
\n
C.
\n
D.
\n
Answer:
\n
Type(문제유형) : 객관식 문제
\n
Page(문제가 만들어진 페이지) :
\n
================================================================================
\n
Question:
\n
Answer:
\n
Type(문제유형) : 단답형 문제
\n
Page(문제가 만들어진 페이지) :
\n
================================================================================
\n
Question:
\n
Answer:
\n
Type(문제유형) : 단답형 문제
\n
Page(문제가 만들어진 페이지) :
\n
<생성 조건>
\n
반드시 위의 예시 형태에 맞춰서 입력받은 PDF에서 영어로 문제를 생성해줘. PDF를 입력받지 않는 경우는 없어.
\n
PDF당 객관식 문제 2개, 단답형 문제 2개
\n
생성한 문제는 텍스트로 출력해줘
\n
객관식은 객관식끼리, 단답형은 단답형끼리
\n
Page는 pdf 제목에 몇페이지부터 몇페이지까지의 pdf인지 나와있어. 참고해줘."""

In [23]:
print(prompt_text)

<문제 생성 예시>


Question : 


Option:


A.


B.


C.


D.


Answer:


Type(문제유형) : 객관식 문제


Page(문제가 만들어진 페이지) :




Question :


Option:


A.


B.


C.


D.


Answer:


Type(문제유형) : 객관식 문제


Page(문제가 만들어진 페이지) :




Question:


Answer:


Type(문제유형) : 단답형 문제


Page(문제가 만들어진 페이지) :




Question:


Answer:


Type(문제유형) : 단답형 문제


Page(문제가 만들어진 페이지) :


<생성 조건>


반드시 위의 예시 형태에 맞춰서 입력받은 PDF에서 영어로 문제를 생성해줘. PDF를 입력받지 않는 경우는 없어.


PDF당 객관식 문제 2개, 단답형 문제 2개


생성한 문제는 텍스트로 출력해줘


객관식은 객관식끼리, 단답형은 단답형끼리


Page는 pdf 제목에 몇페이지부터 몇페이지까지의 pdf인지 나와있어. 참고해줘.


In [22]:
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


i = 0
for file_path in file_paths:
    if i == 6:
        driver.get('https://chatgpt.com/')
        sleep(5)
        i =0
    i = i + 1
    absolute_path = os.path.abspath(file_path)

    # 숨겨진 파일 input 요소 찾기
    file_input = driver.find_element(By.CSS_SELECTOR, 'input[type="file"]')
    try:
        driver.execute_script("arguments[0].style.display = 'block';", file_input)
    except:
        print('파일 업로드 input을 찾을 수 없습니다.')
        continue
    driver.execute_script("arguments[0].value = '';", file_input)
    file_input.send_keys(absolute_path)

    # input을 다시 숨김
    driver.execute_script("arguments[0].style.display = 'none';", file_input)

    # 파일 업로드 완료 대기
    wait = WebDriverWait(driver, 30)  # 30초 타임아웃
    sleep(5)

    # 파일 업로드 진행 중 표시가 사라질 때까지 대기
    try:
        # 업로드 완료 후 전송 버튼이 활성화될 때까지 대기
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="send-button"]')))
    except:
        print('\n====================================')
        print(file_path)
        print("파일 업로드 시간 초과")
        print('====================================\n')
        continue
    sleep(5)

    # 프롬프트 입력
    prompt_text = prompt_text
    prompt_div = driver.find_element(By.ID, "prompt-textarea")

    # JavaScript를 사용하여 텍스트 입력
    driver.execute_script("""
        var element = arguments[0];
        element.innerHTML = arguments[1];
    """, prompt_div, prompt_text)
    sleep(3)

    # 전송 버튼 찾기와 클릭
    send_button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="send-button"]')
    send_button.click()
    print('생성중...')
    sleep(30)

    # 마지막 응답 찾기
    print('생성이 완료되어있어야 함.')
    # ChatGPT의 응답은 일반적으로 role="presentation" 속성을 가진 div 안에 있습니다
    responses = driver.find_elements(By.CSS_SELECTOR, 'div[data-message-author-role="assistant"]')
    last_response = responses[-1]  # 가장 마지막 응답

    # hr 태그를 찾아서 구분선으로 바꾸기
    hr_elements = last_response.find_elements(By.TAG_NAME, 'hr')
    for hr in hr_elements:
        driver.execute_script("""
            var element = arguments[0];
            var separator = '=' * 100 + '\\n';
            element.outerHTML = separator;
        """, hr)

    # 수정된 텍스트 추출
    response_text = last_response.text

    # 나머지 파일 저장 로직
    pdf_filename = os.path.basename(file_path)
    base_name = os.path.splitext(pdf_filename)[0]
    book_name = file_path.split('\\')[-3]
    chap_name = file_path.split('\\')[-2]
    output_filename = f"{book_name}\\{chap_name}\\{base_name}_questions.txt"

    directory = os.path.dirname(output_filename)
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(response_text)

    print(f"응답이 {output_filename}에 저장되었습니다.")

생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_1_to_5_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_4_to_8_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_7_to_11_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_10_to_14_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_13_to_17_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_16_to_20_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_19_to_23_questions.txt에 저장되었습니다.
생성중...
생성이 완료되어있어야 함.
응답이 서강대학교-생명과환경\L&E-14+Ch10-11+Evolution\L&E-14+Ch10-11+Evolution_pages_22_to_25_questions.txt

## cf) GPT 마지막 대화만 저장하는 법

In [4]:
# ChatGPT의 응답은 일반적으로 role="presentation" 속성을 가진 div 안에 있습니다
responses = driver.find_elements(By.CSS_SELECTOR, 'div[data-message-author-role="assistant"]')
last_response = responses[-1]  # 가장 마지막 응답
for i in responses:
    print(i.text)
    print('end')
    print()
    print()

# hr 태그를 찾아서 구분선으로 바꾸기
hr_elements = last_response.find_elements(By.TAG_NAME, 'hr')
for hr in hr_elements:
    driver.execute_script("""
        var element = arguments[0];
        var separator = '=' * 100 + '\\n';
        element.outerHTML = separator;
    """, hr)

# 수정된 텍스트 추출
response_text = last_response.text
print(response_text)
sleep(5)

Here are the questions generated from the provided PDF, formatted as requested:
객관식 문제 (Multiple Choice Questions)
Question 1:
What does "pathogen" mean?
Option:
A. An organism that helps other organisms grow
B. An organism that causes disease
C. A harmless microorganism
D. A component of the immune system
Answer: B
Type(문제유형) : 객관식 문제
Page(문제가 만들어진 페이지) : Pages 1-5
Question 2:
Which of the following statements about Ebola is true?
Option:
A. Ebola is highly contagious and spreads easily before symptoms appear.
B. Ebola requires a large amount of virus particles to infect someone.
C. Ebola is highly infectious but not very contagious.
D. Ebola is not infectious if symptoms do not present themselves.
Answer: C
Type(문제유형) : 객관식 문제
Page(문제가 만들어진 페이지) : Pages 1-5
단답형 문제 (Short Answer Questions)
Question 1:
Define "infectious" as described in the context of the immune system.
Answer: How much of a virus is needed for an infection to occur.
Type(문제유형) : 단답형 문제
Page(문제가 만들어진 페이지) : Pages 1-5
